In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import cv2
import tqdm
from IPython.display import Image
%matplotlib inline

# Importing Training Data


In [ ]:
data = pd.read_csv('../input/identify-the-dance-form/train.csv')

In [ ]:
data.describe()

# Class Distribution

In [ ]:
sns.set()
data.target.value_counts().plot(kind='bar', y='Count', colormap='Paired').set_title('Dance Forms')

In [ ]:
data.target.value_counts().plot(figsize=(20,10))

### *Target Variable is evenly distributed, except for Manipuri having the lowest frequency*

In [ ]:
y = data.target

## Reading a sample image

In [ ]:
Image(filename='../input/identify-the-dance-form/train/1.jpg') 

In [ ]:
Image(filename='../input/identify-the-dance-form/train/10.jpg') 

In [ ]:
Image(filename='../input/identify-the-dance-form/train/12.jpg') 

# Importing image into the dataframe

In [ ]:
X = []
y = []
j=0
path='/kaggle/input/identify-the-dance-form/train'
for img in tqdm.tqdm(data['Image']):
    img_path = os.path.join(path,img)
    img = cv2.imread(img_path)
    X.append(img)
    y.append(data['target'][j])
    j = j+1

In [ ]:
data['Actual_Image'] = X

In [ ]:
type(data.iloc[1,2]),data.iloc[1,2].shape

In [ ]:
type(data.iloc[1,2]),data.iloc[18,2].shape

***We are dealing images of varying sizes of RGB colour format. But for working with this dataset, we need to resize all the images into a constant dimension**

## Plotting images from each category

In [ ]:
data_group = data.groupby('target', as_index=False)

In [ ]:
rows, columns = (4,2)
fig=plt.figure(figsize=(20, 20))
i = 1
for group_name, group_df in data_group:
    fig.add_subplot(rows, columns, i)
    group_df = group_df.reset_index()
    plt.imshow(group_df['Actual_Image'][np.random.randint(low=0, high=len(group_df))])
    plt.gca().set_title(group_name)
    i = i+1
plt.show()

In [ ]:
rows, columns = (4,2)
fig=plt.figure(figsize=(20, 20))
i = 1
for group_name, group_df in data_group:
    fig.add_subplot(rows, columns, i)
    group_df = group_df.reset_index()
    plt.imshow(group_df['Actual_Image'][np.random.randint(low=0, high=len(group_df))])
    plt.gca().set_title(group_name)
    i = i+1
plt.show()

# Applying K Nearest Neighbours

## For applying various algorithms, we require to have the images of same dimensions

In [ ]:
img_height, img_width = (224,224)

In [ ]:
data['Actual_Image'] = data['Actual_Image'].apply(lambda x : cv2.resize(x,(img_height, img_width)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['Actual_Image'].apply(lambda x: x.flatten()), data['target'], test_size=0.3, random_state=42 )

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

*Need to change the shape of X_train*

In [ ]:
arr = []
for each in X_train:
    arr.append(each)
arr = np.array(arr)


In [ ]:
arr.shape

In [ ]:
X_train = arr

In [ ]:
arr = []
for each in X_test:
    arr.append(each)
arr = np.array(arr)

In [ ]:
arr.shape

In [ ]:
X_test = arr

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Building and Training the Model

### Simplest Model Possible : K Nearest Neighbours 

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn_model.fit(X_train, y_train)

## Accuracy Score is 🙁

In [ ]:
knn_model.score(X_test, y_test)

## Trying out with different values of k

In [ ]:
accuracy_score_list = []
for k in tqdm.tqdm(range(3, 50)):
    knn_model = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    knn_model.fit(X_train, y_train)
    accuracy_score_list.append(knn_model.score(X_test, y_test))


## K Vs Accuracy

In [ ]:
sns.lineplot(range(3,50), accuracy_score_list)
plt.title("K Vs Accuracy")
plt.xlabel("K")
plt.ylabel("Accuracy")
plt.show()

## So, 30% is the highest accuracy with this basic model, which occurs at k=4

### Since the problem is an image classificatin problem, such distance based simple ML Models won't work well. Thus, our next approach will be Convolutional Neural Network